In [0]:
import os

!git clone https://github.com/Sontref/CycleGAN.git
os.chdir('CycleGAN')
os.mkdir('generated_images/water2wine')
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ic8IGYC45pzRylqec550oV7ISTNDxXeJ' -O water2wine.zip
!unzip -q water2wine.zip -d datasets/

Uncomment and run this script for train model.
--start from 0 will train it from scratch;
--start from 180 will train it from last saved state (180-th epoch).
If you want just play with models, skip this line.

In [0]:
#!python train.py --dataset_name water2wine --img_height 128 --img_width 128 --batch_size 5 --num_residual 6 --start_from 180

In [28]:
from datasets import ImageDataset
from model import Generator

import torch
from torch.utils.data import DataLoader

from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt

device = 'cuda'

<Figure size 1440x720 with 0 Axes>

In [0]:
channels = 3
num_residual = 6
start_from = 180
dataset_name = "water2wine"

test_loader = DataLoader(
    ImageDataset(path_to_data='datasets/%s' % dataset_name, size=(128,128), mode='test'),
    batch_size=5,
    num_workers=2,
    shuffle=True
)

G_AB = Generator(in_channels=channels, num_residual=num_residual).to(device=device)
G_BA = Generator(in_channels=channels, num_residual=num_residual).to(device=device)
G_AB.load_state_dict(torch.load("saved_models/%s/G_AB_%d.pth" % (dataset_name, 180)))
G_BA.load_state_dict(torch.load("saved_models/%s/G_BA_%d.pth" % (dataset_name, 180)))

def sample_images():
    """Saves a generated sample from the test set"""
    imgs = next(iter(test_loader))
    G_AB.eval()
    G_BA.eval()
    real_A = imgs["A"].permute(0, 3, 1, 2).to(device=device).float()
    fake_B = G_AB(real_A)
    real_B = imgs["B"].permute(0, 3, 1, 2).to(device=device).float()
    fake_A = G_BA(real_B)
    # Arange images along x-axis
    real_A = make_grid(real_A, nrow=5, normalize=True)
    real_B = make_grid(real_B, nrow=5, normalize=True)
    fake_A = make_grid(fake_A, nrow=5, normalize=True)
    fake_B = make_grid(fake_B, nrow=5, normalize=True)
    # Arange images along y-axis
    image_grid = torch.cat((real_A, fake_B, real_B, fake_A), 1).permute(1, 2, 0)

    plt.figure(figsize=(20,10))
    plt.imshow(image_grid.detach().cpu().numpy())

In [0]:
sample_images()